### start

In [24]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import gc
import time
import copy
import logging
import re
from itertools import chain,combinations
import pdb
import math
from pathlib import Path
import datetime
from datetime import datetime,timedelta
import json
import warnings
import yaml
from typing import Dict,Union,List,Any,Tuple
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import IPython
import requests
import warnings
import boto3
import matplotlib as plt
import sklearn

# load_dotenv(Path("../../../Local/.env"))

# Custom format function for displaying |numbers/
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')

# Suppress warnings
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings('ignore', message='.*frozen modules.*')
warnings.filterwarnings("ignore", message="MallocStackLogging")

# silence pygame donation request
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"
os.environ['LOGGING_FILE'] = "../../../Local/logs/wallet_modeling.log"
os.environ['NOTIFICATION_SOUNDS_DIR'] = "../../../Local"

# Dark mode charts
plt.rcParams['figure.facecolor'] = '#181818'  # Custom background color (dark gray in this case)
plt.rcParams['axes.facecolor'] = '#181818'
plt.rcParams['text.color'] = '#afc6ba'
plt.rcParams['axes.labelcolor'] = '#afc6ba'
plt.rcParams['xtick.color'] = '#afc6ba'
plt.rcParams['ytick.color'] = '#afc6ba'
plt.rcParams['axes.titlecolor'] = '#afc6ba'
plt.rcParams['text.usetex'] = False
plt.rcParams['mathtext.default'] = 'regular'

# import local modules
# pyright: reportMissingImports=false
sys.path.append('../src')

import sage_utils.config_validation as ucv
import sage_utils.s3_utils as s3u

import sage_wallet_insights.model_evaluation as sime

import sage_wallet_modeling.wallet_modeler as wm
from sage_wallet_modeling.wallet_preprocessor import SageWalletsPreprocessor
import sage_wallet_modeling.wallet_script_modeler as sm
import sage_wallet_modeling.workflow_orchestrator as wo

import script_modeling.entry_helpers as h
# import script_modeling.temporal_cv_model_entry as tme


# import data-science modules
sys.path.append(str(Path("..") / ".." / "data-science" / "src"))
import wallet_insights.model_evaluation as wime
import utils as u
from utils import ConfigError



# reload all modules
modules = [
    ucv, s3u,
    sime,
    wm, sm, wo,
    h,
    wime,
    u,
]

# import utils as u
# Set the custom error handler
ipython = IPython.get_ipython()
# ipython.set_custom_exc((Exception,), u.notify_on_failure)

player = u.AmbientPlayer()
player.stop_all_players()

# configure logger
logger = u.setup_notebook_logger('../logs/notebook_logs.log')
logger.setLevel(logging.INFO)

from IPython.core import ultratb
ultratb.VerboseTB._tb_highlight = "bg:#b45827"


# load all configs
sage_wallets_config = ucv.load_sage_wallets_config(Path('../config/sage_wallets_config.yaml'))
sage_wallets_modeling_config = ucv.load_sage_wallets_modeling_config(Path('../config/sage_wallets_modeling_config.yaml'))


u.export_code(
    code_directories=[
        # 'sage_wallet_insights',
        'sage_wallet_modeling',
        # 'sage_utils',
    ],
    # include_config = True,
    # ipynb_notebook = 'DDA-769 coin model score dist toggle.ipynb'
    output_file="temp/sagemaker_code.py"
)

[importlib.reload(module) for module in modules]
u.notify('retro')

logger.milestone("Good morning, let's get to work")

[02/Aug/25 11:40:13] INFO [utils.export_code:1659] Consolidation complete. All files are saved in temp/sagemaker_code.py
[02/Aug/25 11:40:13] MILESTONE [472202081.<module>:134] Good morning, let's get to work


# Model Training

## Multi Model Sequence

### Initiate orchestrator

In [12]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '230908',
    '231008',
    '231107',
    '231207',
    '240106',
    '240205',
    '240306',
    '240405',
    '240505',
    '240604',
    '240704',
    '240803',
    '240902',
    '241002',
    '241201',
    '241101',
    '241231',
    '250130',
    '250301',
]
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)

### Load, preprocess, and upload all data

#### load data

In [13]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

workflow_orch.load_all_training_data(date_suffixes)

[01/Aug/25 23:38:13] MILESTONE [workflow_orchestrator.load_all_training_data:123] <PROD> Loading training data for 19 periods: ['230908', '231008', '231107', '231207', '240106', '240205', '240306', '240405', '240505', '240604', '240704', '240803', '240902', '241002', '241201', '241101', '241231', '250130', '250301']


[01/Aug/25 23:38:16] INFO [workflow_orchestrator.load_all_training_data:139] Training data loaded successfully: 4,617,462 total rows and 9 offsets for each date_suffix.


##### inspect features

In [10]:
# Create combined NaN count and describe statistics
first_key = next(iter(workflow_orch.training_data))
nan_counts = workflow_orch.training_data[first_key]['x_train'].isna().sum()
describe_stats = workflow_orch.training_data[first_key]['x_train'].describe().T

# Combine into single DataFrame
combined_stats = pd.concat([
    nan_counts.rename('nan_count'),
    describe_stats
], axis=1).sort_index()

u.display_full(combined_stats.sort_index())

,nan_count,count,mean,std,min,25%,50%,75%,max
cluster|k5_cluster_k0,0,19894,0.248768472906,0.432310205975,0,0,0,0,1
cluster|k5_cluster_k1,0,19894,0.194681813612,0.39596551146,0,0,0,0,1
cluster|k5_cluster_k2,0,19894,0.210666532623,0.407792231024,0,0,0,0,1
cluster|k5_cluster_k3,0,19894,0.14089675279,0.347923759809,0,0,0,0,1
cluster|k5_cluster_k4,0,19894,0.204986428069,0.403701851066,0,0,0,0,1
cw_cluster|k5_cluster_k0,0,19894,0.345279983915,0.475471429747,0,0,0,1,1
cw_cluster|k5_cluster_k1,0,19894,0.151201367246,0.358254051311,0,0,0,0,1
cw_cluster|k5_cluster_k2,0,19894,0.151402432894,0.358449710208,0,0,0,0,1
cw_cluster|k5_cluster_k3,0,19894,0.155876143561,0.362746999649,0,0,0,0,1
cw_cluster|k5_cluster_k4,0,19894,0.196240072384,0.397162227924,0,0,0,0,1


#### preprocess all data

In [14]:
workflow_orch.preprocess_all_training_data()

[01/Aug/25 23:38:17] INFO [workflow_orchestrator.preprocess_all_training_data:170] Preprocessing 19 date periods...
[01/Aug/25 23:38:17] INFO [wallet_preprocessor.preprocess_training_data:77] Starting preprocessing for SageMaker XGBoost compatibility...
[01/Aug/25 23:38:17] INFO [wallet_preprocessor._handle_missing_values:237] Filled NaN values in 100 columns for x_train.
[01/Aug/25 23:38:17] INFO [wallet_preprocessor.preprocess_y_data:276] Applied classification threshold 0.05 to y_train: 5,301 positive (26.6%), 14,593 negative (73.4%)
[01/Aug/25 23:38:17] INFO [wallet_preprocessor._combine_x_y_data:387] Merged y df with target var cw_crypto_net_gain/crypto_inflows/winsorized with X data.
[01/Aug/25 23:38:19] INFO [wallet_preprocessor._save_preprocessed_df:433] Saved preprocessed train split to ../s3_uploads/wallet_training_data_preprocessed/dda_927_concat_offsets/230908/train.csv
[01/Aug/25 23:38:19] INFO [wallet_preprocessor.preprocess_training_data:119] Preprocessed train: 19,894 r

#### upload all data

In [ ]:
# Upload and retrieve URIs
workflow_orch.upload_all_training_data(overwrite_existing=True)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

#### concatenate and upload and model

In [15]:
workflow_orch.concatenate_all_preprocessed_data()

[01/Aug/25 23:40:48] INFO [workflow_orchestrator.concatenate_all_preprocessed_data:201] Beginning concatenation of preprocessed data...
[01/Aug/25 23:42:20] INFO [workflow_orchestrator.concatenate_all_preprocessed_data:245] Saved concatenated train.csv with 1192664 rows to ../s3_uploads/wallet_training_data_concatenated/dda_927_concat_offsets/train.csv
[01/Aug/25 23:42:24] INFO [workflow_orchestrator.concatenate_all_preprocessed_data:245] Saved concatenated eval.csv with 74553 rows to ../s3_uploads/wallet_training_data_concatenated/dda_927_concat_offsets/eval.csv
[01/Aug/25 23:42:38] INFO [workflow_orchestrator.concatenate_all_preprocessed_data:245] Saved concatenated test.csv with 223637 rows to ../s3_uploads/wallet_training_data_concatenated/dda_927_concat_offsets/test.csv


In [16]:
concat_uris = workflow_orch.upload_concatenated_training_data(overwrite_existing=True)

[01/Aug/25 23:42:38] INFO [workflow_orchestrator.upload_concatenated_training_data:279] Beginning upload of concatenated training data...
[02/Aug/25 00:09:40] INFO [workflow_orchestrator.upload_concatenated_training_data:308] Uploaded concatenated split 'train' to s3://wallet-training-data/training-data-concatenated/dda-928-concat-allo/train.csv
[02/Aug/25 00:11:22] INFO [workflow_orchestrator.upload_concatenated_training_data:308] Uploaded concatenated split 'eval' to s3://wallet-training-data/training-data-concatenated/dda-928-concat-allo/eval.csv
[02/Aug/25 00:16:26] INFO [workflow_orchestrator.upload_concatenated_training_data:308] Uploaded concatenated split 'test' to s3://wallet-training-data/training-data-concatenated/dda-928-concat-allo/test.csv
[02/Aug/25 00:16:26] INFO [workflow_orchestrator.None:248] (2028.1s) Completed upload_concatenated_training_data.


In [17]:
workflow_orch.train_concatenated_offsets_model()

[02/Aug/25 00:16:26] INFO [workflow_orchestrator.concatenate_all_preprocessed_data:201] Beginning concatenation of preprocessed data...
[02/Aug/25 00:17:58] INFO [workflow_orchestrator.concatenate_all_preprocessed_data:245] Saved concatenated train.csv with 1192664 rows to ../s3_uploads/wallet_training_data_concatenated/dda_927_concat_offsets/train.csv
[02/Aug/25 00:18:02] INFO [workflow_orchestrator.concatenate_all_preprocessed_data:245] Saved concatenated eval.csv with 74553 rows to ../s3_uploads/wallet_training_data_concatenated/dda_927_concat_offsets/eval.csv
[02/Aug/25 00:18:16] INFO [workflow_orchestrator.concatenate_all_preprocessed_data:245] Saved concatenated test.csv with 223637 rows to ../s3_uploads/wallet_training_data_concatenated/dda_927_concat_offsets/test.csv
[02/Aug/25 00:18:16] INFO [workflow_orchestrator.upload_concatenated_training_data:279] Beginning upload of concatenated training data...
[02/Aug/25 00:18:27] INFO [workflow_orchestrator.upload_concatenated_trainin

2025-08-02 07:18:31 Starting - Starting the training job...
2025-08-02 07:18:46 Starting - Preparing the instances for training...
2025-08-02 07:19:07 Downloading - Downloading input data...
2025-08-02 07:20:03 Training - Training image download completed. Training in progress...../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-08-02 07:20:38.517 ip-10-0-228-113.us-west-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-08-02 07:20:38.536 ip-10-0-228-113.us-west-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-08-02:07:20:38:INFO] Imported framework sagemaker_xgboost_container.training
[2025-08-02:07:20:38:INFO] No GPUs d

[02/Aug/25 00:40:17] INFO [wallet_script_modeler.train_single_period_script_model:129] Script-mode training completed. Model URI: s3://wallet-script-models/model-outputs/dda-928-concat-allo/concat/wscript-dda-928-concat-allo-concat-20250802-001830/output/model.tar.gz
[02/Aug/25 00:40:17] INFO [workflow_orchestrator.None:632] (1431.0s) Completed train_concatenated_offsets_model.


{'model_uri': 's3://wallet-script-models/model-outputs/dda-928-concat-allo/concat/wscript-dda-928-concat-allo-concat-20250802-001830/output/model.tar.gz',
 'training_job_name': 'wscript-dda-928-concat-allo-concat-20250802-001830',
 'date_suffix': 'concat'}

In [22]:
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
s3_uris = workflow_orch.retrieve_training_data_uris(['concat'])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_     │
│ ❱ 2 s3_uris = workflow_orch.retrieve_training_data_uris(['concat'])                              │
│   3                                                                                              │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/GitHub/sagemaker/notebooks/../src/sage_wallet_modeling/workflow_o │
│ rchestrator.py:403 in retrieve_training_data_uris                                                │
│                                                                                                  │
│    400 │   │   │   │   │   )                                                                     │
│    401 │   │   │   │   │                                                                         │
│    402 │   │   │   │   │   if 'Contents' not in response:                                        │
│ ❱  403 │   │   │   │   │   │   raise FileNotFoundError("No S3 objects found matching prefix: "   │
│    404 │   │   │   │   │   │   │   │   │   │   │   │   f"s3://{bucket_name}/{prefix}")           │
│    405 │   │   │   │   │                                                                         │
│    406 │   │   │   │   │   # Filter for CSV files that start with the exact split name           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: No S3 objects found matching prefix: 
s3://wallet-training-data/training-data-preprocessed/dda-928-concat-allo/concat/train

In [25]:
# =============================================================================
# Concatenated Model Prediction Pipeline
# =============================================================================

wallets_config = sage_wallets_config
modeling_config = sage_wallets_modeling_config

# Initialize the workflow orchestrator
from sage_wallet_modeling.workflow_orchestrator import WalletWorkflowOrchestrator

orchestrator = WalletWorkflowOrchestrator(
    wallets_config=wallets_config,
    modeling_config=modeling_config
)

# Check prerequisites - ensure concatenated model exists and test data is uploaded
print("Checking prerequisites...")

# Verify concatenated test data exists in S3
bucket = wallets_config['aws']['training_bucket']
base_folder = wallets_config['aws']['concatenated_directory']
upload_directory = wallets_config['training_data']['upload_directory']
dataset = wallets_config['training_data'].get('dataset', 'dev')
if dataset == 'dev':
    upload_directory = f"{upload_directory}-dev"

test_s3_uri = f"s3://{bucket}/{base_folder}/{upload_directory}/test.csv"
print(f"Expected test data location: {test_s3_uri}")

# Generate predictions for concatenated test data
print("\nLaunching concatenated model prediction...")
try:
    prediction_results = orchestrator.predict_concatenated_test_data(
        overwrite_existing=False  # Set to True if you want to regenerate predictions
    )

    print("✅ Prediction completed successfully!")
    print(f"Transform job: {prediction_results.get('transform_job_name', 'existing file used')}")
    print(f"Predictions URI: {prediction_results['predictions_uri']}")

    # If you want to see where the predictions were downloaded locally
    if 'predictions_uri' in prediction_results:
        local_path = (f"{wallets_config['training_data']['local_s3_root']}/"
                     f"s3_downloads/wallet_predictions/"
                     f"{wallets_config['training_data']['local_directory']}/"
                     f"concat/test.csv.out")
        print(f"Local predictions path: {local_path}")

except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("\nTroubleshooting steps:")
    print("1. Run concatenate_all_preprocessed_data() to create concatenated CSVs")
    print("2. Run upload_concatenated_training_data() to upload test data to S3")
    print("3. Run train_concatenated_offsets_model() to train the model")

except Exception as e:
    print(f"❌ Unexpected error: {e}")
    raise

Checking prerequisites...
Expected test data location: s3://wallet-training-data/training-data-concatenated/dda-928-concat-allo/test.csv

Launching concatenated model prediction...
❌ Unexpected error: 'WalletWorkflowOrchestrator' object has no attribute 'predict_concatenated_test_data'


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:33                                                                                   │
│                                                                                                  │
│   30 # Generate predictions for concatenated test data                                           │
│   31 print("\nLaunching concatenated model prediction...")                                       │
│   32 try:                                                                                        │
│ ❱ 33 │   prediction_results = orchestrator.predict_concatenated_test_data(                       │
│   34 │   │   overwrite_existing=False  # Set to True if you want to regenerate predictions       │
│   35 │   )                                                                                       │
│   36                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'WalletWorkflowOrchestrator' object has no attribute 'predict_concatenated_test_data'

In [20]:
# Launch modeler
modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    'concat',
    s3_uris
)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│   3 │   sage_wallets_config,                                                                     │
│   4 │   sage_wallets_modeling_config,                                                            │
│   5 │   'concat',                                                                                │
│ ❱ 6 │   s3_uris                                                                                  │
│   7 )                                                                                            │
│   8                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 's3_uris' is not defined

### Train all models

In [ ]:
workflow_orch.load_all_training_data(date_suffixes)
modeling_results = workflow_orch.train_all_models()

### Predict with all models

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
workflow_orch.load_all_training_data(date_suffixes)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

In [ ]:
result = workflow_orch.predict_with_all_models()

### Evaluate all models

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
workflow_orch.load_all_training_data(date_suffixes)

evaluators = workflow_orch.evaluate_all_models()


In [ ]:
for e in evaluators:
    evaluators[e].plot_wallet_evaluation()

### DDA 923 temporal cv model

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '241101',
    '250301'
]
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
workflow_orch.load_all_training_data(date_suffixes)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

In [ ]:
sm.train_temporal_cv_script_model(sage_wallets_config,sage_wallets_modeling_config,cv_s3_uri)

## Single Model Sequence

### Train Single Model

#### retrieve file uris

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)



date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]

# Generate URIs for specific dates without any prior setup
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)
s3_uris

#### train model

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffix = '231008'

modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    s3_uris
)



In [ ]:
modeling_results = modeler.train_model()

### Score validation set

#### get existing model uri

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))



date_suffix = '231008'

# Generate URIs for specific dates without any prior setup
workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
s3_uris = workflow_orch.retrieve_training_data_uris([date_suffix])

# Launch modeler
modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    s3_uris
)


model_metadata = modeler.load_existing_model()

#### predict with batch transform

In [ ]:
transform_results = modeler.predict_with_batch_transform('val')

In [ ]:
transform_results = modeler.predict_with_batch_transform('test')

### Predict Using Endpoints

#### deploy endpoint for predictions

In [ ]:
modeler.list_active_endpoints()

In [ ]:
modeler.deploy_endpoint()

#### load and preprocess test/val dfs

In [ ]:
x_test = pd.read_parquet(
    Path("../s3_uploads/wallet_training_data_queue") /
    sage_wallets_config['training_data']['local_directory'] /
    f"x_test_{date_suffix}.parquet"
)
x_val = pd.read_parquet(
    Path("../s3_uploads/wallet_training_data_queue") /
    sage_wallets_config['training_data']['local_directory'] /
    f"x_val_{date_suffix}.parquet"
)




In [ ]:
preprocessor = SageWalletsPreprocessor(sage_wallets_config)
x_test_processed = preprocessor.preprocess_x_df(x_test)
x_val_processed = preprocessor.preprocess_x_df(x_val)

logger.info(f"Preprocessed X_test {x_test_processed.shape} and "
            f"X_val {x_val_processed.shape}.")

#### predict

In [ ]:
y_test_pred = modeler.predict_using_endpoint(x_test_processed, 'test')
y_val_pred = modeler.predict_using_endpoint(x_val_processed, 'val')

#### delete all endpoints

In [ ]:
modeler.delete_all_endpoints()

In [ ]:
modeler.list_active_endpoints()
modeler.list_all_endpoints()

## Script model training

In [6]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    '231107',
    '240306',
    '250301'
]

date_suffix = '231008'


workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_config)
workflow_orch.load_all_training_data(date_suffixes)
s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)

[01/Aug/25 23:13:11] MILESTONE [workflow_orchestrator.load_all_training_data:123] <PROD> Loading training data for 4 periods: ['231008', '231107', '240306', '250301']
[01/Aug/25 23:13:11] INFO [workflow_orchestrator.load_all_training_data:139] Training data loaded successfully: 745,742 total rows and 9 offsets for each date_suffix.
[01/Aug/25 23:13:11] INFO [credentials.load:1352] Found credentials in shared credentials file: ~/.aws/credentials


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:18                                                                                   │
│                                                                                                  │
│   15                                                                                             │
│   16 workflow_orch = wo.WalletWorkflowOrchestrator(sage_wallets_config,sage_wallets_modeling_    │
│   17 workflow_orch.load_all_training_data(date_suffixes)                                         │
│ ❱ 18 s3_uris = workflow_orch.retrieve_training_data_uris(date_suffixes)                          │
│   19                                                                                             │
│                                                                                                  │
│ /Users/jeremymeadow/DreamsData/GitHub/sagemaker/notebooks/../src/sage_wallet_modeling/workflow_o │
│ rchestrator.py:403 in retrieve_training_data_uris                                                │
│                                                                                                  │
│    400 │   │   │   │   │   )                                                                     │
│    401 │   │   │   │   │                                                                         │
│    402 │   │   │   │   │   if 'Contents' not in response:                                        │
│ ❱  403 │   │   │   │   │   │   raise FileNotFoundError("No S3 objects found matching prefix: "   │
│    404 │   │   │   │   │   │   │   │   │   │   │   │   f"s3://{bucket_name}/{prefix}")           │
│    405 │   │   │   │   │                                                                         │
│    406 │   │   │   │   │   # Filter for CSV files that start with the exact split name           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: No S3 objects found matching prefix: 
s3://wallet-training-data/training-data-preprocessed/dda-928-concat-pipe/231008/train

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))

date_suffix = '231008'

modeler = wm.WalletModeler(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    s3_uris
)



In [ ]:
sm.train_single_period_script_model(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    s3_uris
)

In [ ]:
from sagemaker.session import Session
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter, IntegerParameter

s3_data = {
    "train":      s3_uris[date_suffix]['train'],
    "validation": s3_uris[date_suffix]['val']
}
model_output_path = (f"s3://wallet-script-models/model-outputs")

estimator = XGBoost(
    entry_point         = "script_mode_modeler.py",     # path relative to notebook
    source_dir          = "../src/script_modeling/",    # folder that holds the script
    base_job_name       = "dda-922-v4",                 # uploaded and immutable script version
    framework_version   = sage_wallets_modeling_config['framework']['version'],
    role                = sage_wallets_config['aws']['modeler_arn'],
    instance_type       = sage_wallets_modeling_config['metaparams']['instance_type'],
    instance_count      = sage_wallets_modeling_config['metaparams']['instance_count'],
    output_path         = model_output_path,
    hyperparameters = {
        "eta": 0.1,
        "max_depth": 6,
        "subsample": 1.0,
        "num_boost_round": 50,            # small for a smoke-test
        "early_stopping_rounds": 10
    },
    # metric_definitions=[                  # regex must match the print line
    #     {"Name": "cv_auc_pr", "Regex": "validation:cv_auc_pr=(.*)"}
    # ],
)

# one-shot fit (no tuning yet)
estimator.fit(s3_data, logs="All")

# Model Evaluation

### Analyze predictions

In [ ]:
[importlib.reload(module) for module in modules]
sage_wallets_config = yaml.safe_load(Path('../config/sage_wallets_config.yaml').read_text(encoding='utf-8'))
sage_wallets_modeling_config = yaml.safe_load(Path('../config/sage_wallets_modeling_config.yaml').read_text(encoding='utf-8'))


date_suffixes = [
    '231008',
    # '240306'
]
date_suffix = date_suffixes[0]

In [ ]:
y_test_pred = sime.load_bt_sagemaker_predictions('test', sage_wallets_config, date_suffix)
y_val_pred = sime.load_bt_sagemaker_predictions('val', sage_wallets_config, date_suffix)

In [ ]:
# Single function call for complete evaluation
evaluator = sime.run_sagemaker_evaluation(
    sage_wallets_config,
    sage_wallets_modeling_config,
    date_suffix,
    y_test_pred,
    y_val_pred
)

In [ ]:
evaluator.y_validation

In [ ]:
evaluator.y_validation_pred_proba

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(evaluator.y_validation, evaluator.y_validation_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Validation Set')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
evaluator.summary_report()
evaluator.plot_wallet_evaluation()


In [ ]:
def load_sagemaker_predictions(
    data_type: str,
    sage_wallets_config: dict,
    sage_wallets_modeling_config: dict,
    date_suffix: str
) -> tuple[pd.Series, pd.Series]:
    """
    Load SageMaker predictions and corresponding actuals for a given data type.

    Params:
    - data_type (str): Either 'test' or 'val'
    - sage_wallets_config (dict): Configuration for training data paths
    - sage_wallets_modeling_config (dict): Configuration for model parameters
    - date_suffix (str): Date suffix for file naming

    Returns:
    - tuple: (predictions_series, actuals_series) with aligned indices
    """
    # Load predictions
    pred_path = Path(sage_wallets_modeling_config['metaparams']['endpoint_preds_dir']) / \
                f"endpoint_y_pred_{data_type}_{sage_wallets_config['training_data']['local_directory']}_{date_suffix}.csv"
    pred_df = pd.read_csv(pred_path)

    if 'score' not in pred_df.columns:
        raise ValueError(f"SageMaker predictions are missing the 'score' column. "
                        f"Available columns: {pred_df.columns}")
    pred_series = pred_df['score']

    # Load actuals
    training_data_path = (
        Path(f"../s3_uploads") / "wallet_training_data_queue" /
        f"{sage_wallets_config['training_data']['local_directory']}"
    )
    actuals_path = training_data_path / f"y_{data_type}_{date_suffix}.parquet"
    actuals_df = pd.read_parquet(actuals_path)

    if len(actuals_df.columns) > 1:
        raise ValueError(f"Found unexpected columns in y_{data_type}_df. "
                        f"Expected 1 column, found {actuals_df.columns}.")
    actuals_series = actuals_df.iloc[:, 0]

    # Validate lengths and align indices
    if len(pred_series) != len(actuals_series):
        raise ValueError(f"Length of y_{data_type}_pred ({len(pred_series)}) does "
                        f"not match length of y_{data_type}_true ({len(actuals_series)}).")

    pred_series.index = actuals_series.index

    return pred_series, actuals_series

In [ ]:
# Load predictions and actuals
y_test_pred_series, y_test_true_series = load_sagemaker_predictions(
    'test', sage_wallets_config, sage_wallets_modeling_config, date_suffix
)
y_val_pred_series, y_val_true_series = load_sagemaker_predictions(
    'val', sage_wallets_config, sage_wallets_modeling_config, date_suffix
)

# Load remaining training data
training_data_path = (
    Path(f"../s3_uploads") / "wallet_training_data_queue" /
    f"{sage_wallets_config['training_data']['local_directory']}"
)
X_train = pd.read_parquet(training_data_path / f"x_train_{date_suffix}.parquet")
y_train = pd.read_parquet(training_data_path / f"y_train_{date_suffix}.parquet")
X_test = pd.read_parquet(training_data_path / f"x_test_{date_suffix}.parquet")
X_val = pd.read_parquet(training_data_path / f"x_val_{date_suffix}.parquet")

# Identify target variable and model type
target_variable = y_val_true_series.name or y_train.columns[0]
objective = sage_wallets_modeling_config['training']['hyperparameters']['objective']
model_type = 'regression' if objective[:3] == 'reg' else 'unknown'

In [ ]:
# Create model_id and modeling_config
model_id = f"sagemaker_{sage_wallets_config['training_data']['local_directory']}_{date_suffix}"

modeling_config = {
    'target_variable': target_variable,
    'model_type': model_type,
    'returns_winsorization': 0.005,  # Default for winsorizing returns
    'training_data': {
        'modeling_period_duration': 30  # Default performance window
    },
    'sagemaker_metadata': {
        'objective': objective,
        'local_directory': sage_wallets_config['training_data']['local_directory'],
        'date_suffix': date_suffix
    }
}

# Create minimal wallet_model_results for SageMaker evaluation
wallet_model_results = {
    'model_id': model_id,
    'modeling_config': modeling_config,
    'model_type': model_type,

    # Training data
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test_true_series,
    'y_pred': y_test_pred_series,
    'training_cohort_pred': None,
    'training_cohort_actuals': None,

    # Validation data
    'X_validation': X_val,
    'y_validation': y_val_true_series,
    'y_validation_pred': y_val_pred_series,
    'validation_target_vars_df': None,

    # Fixed mock pipeline
    'pipeline': type('MockPipeline', (), {
        'named_steps': {'estimator': type('MockModel', (), {
            'get_params': lambda self: {'objective': objective}  # Accept self argument
        })()},
        '__getitem__': lambda self, key: type('MockTransformer', (), {
            'transform': lambda self, X: X  # Accept self argument
        })()
    })()
}

# Create evaluator
wallet_evaluator = wime.RegressorEvaluator(wallet_model_results)

# Run basic evaluation
wallet_evaluator.summary_report()
wallet_evaluator.plot_wallet_evaluation()

In [ ]:
target_variable